# F16 Multiagent Environment


This notebook outlines the use of a multiagent F16 air collision avoidance scenario, exposing the control inputs/outputs to an external environment. Two F16s and their low level controller, labeled a and b, are instantiated for this scenario. This setup is centralized, meaning that the agent states and agent actions are exposed.

## Inputs
The system topology inputs are

`time` float - current simulation time

`planta-states`(13,) - state vector for F16 "a"

`controllera-states` (3,) - low level controller integrators for F16 "a" 

`plantb-states` (13,) - state vector for F16 "b"

`controllerb-states` (3,) - low level controller integrators for F16 "b" 

## Outputs

`autopilot-outputsa` (4,) - high level control input for F16 "a"

`autopilot-outputsb` (4,) - high level control input for F16 "b"

## System Configuration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# CSAF Imports
import csaf.config as cconf
import csaf.system as csys

# load the centralized multiagent config
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_multiagent_central_config.toml")

In [ ]:
# view the CSAF system topology
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

## Simulation Config

Steps to configure

1. the collision condition
2. the system environment
3. initial state of each aircraft

In [ ]:
# termination condition -- what *is* an air collision
def air_collision_condition(ctraces):
        """ground collision premature termnation condition
        
        Collision occurs if two aircraft are within l meters of one another
        """
        # get the aircraft states
        sa, sb = ctraces['planta']['states'], ctraces['plantb']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 10

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

# create an environment from the system, allowing us to act as the controller
my_env = csys.SystemEnv("autopilot", my_system, terminating_conditions_all=air_collision_condition)

In [ ]:
# set the scenario states
my_system.set_state('planta', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_system.set_state('plantb', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 2000.0, 0.0, 6000.0, 9.0])

## Run a Simulation

In [ ]:
# collect aircraft states
pstates = []

# send signal of zeros - for this example, send only zeros except for throttle
ctrl_signal = [0.,0.,0.,0.7]

# step through simulation and collect f16 states
# StopIteration is thrown when the terminating conditions are achieved
do_sim = True
while do_sim:
    try:
        # send the outputs, can collect the inputs
        comp_input = my_env.step({"autopilot-outputsa": ctrl_signal, "autopilot-outputsb": ctrl_signal}) 
        
        # get the states and track them over time
        pstates.append((comp_input['planta-states'], comp_input['plantb-states']))
    
    # stop iteration occurs when the termination conditions are satisfied
    except StopIteration as e:
        do_sim = False
        break
        
    # other errors can occur -- maybe solver error
    except Exception as e:
        raise e
        break
        
pstates = np.array(pstates)